## Preprocessing Physionet 2012 dataset

In [ ]:
import os
import tarfile
import pickle
import pandas as pd
import numpy as np
from numpy import ma
import seaborn as sns
from itertools import chain
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.utils import shuffle
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Import Physionet 2012 Challenge files
source: https://www.physionet.org/content/challenge-2012/1.0.0/

In [ ]:
!wget https://physionet.org/files/challenge-2012/1.0.0/set-a.zip
!wget https://physionet.org/files/challenge-2012/1.0.0/set-b.zip

!wget https://physionet.org/files/challenge-2012/1.0.0/Outcomes-a.txt
!wget https://physionet.org/files/challenge-2012/1.0.0/Outcomes-b.txt

!wget https://physionet.org/files/challenge-2012/1.0.0/set-c.tar.gz   
!wget https://physionet.org/files/challenge-2012/1.0.0/Outcomes-c.txt

In [ ]:
!unzip -u set-a.zip
!unzip -u set-b.zip

In [ ]:
datasets = ['set-a', 'set-b']

In [ ]:
def get_dataframe(dataset):
    txt_all = list()
    for f in os.listdir(dataset):
        with open(os.path.join(dataset, f), 'r') as fp:
            txt = fp.readlines()

        # get recordid to add as a column
        recordid = txt[1].rstrip('\n').split(',')[-1]
        txt = [t.rstrip('\n').split(',') + [int(recordid)] for t in txt]
        txt_all.extend(txt[1:])
    df = pd.DataFrame(txt_all, columns=['time', 'parameter', 'value', 'recordid'])
    return df

In [ ]:
df_a = get_dataframe('set-a')
df_b = get_dataframe('set-b')
df = pd.concat([df_a, df_b])
df.reset_index(inplace=True, drop=True) 

In [ ]:
txt_all = list()
tar = tarfile.open("set-c.tar.gz", "r:gz")
for member in tar.getmembers():
    f = tar.extractfile(member)
    if f is not None:
        txt = f.readlines()
    
        txt = [x.decode("utf-8") for x in txt]
        # get recordid to add as a column
        recordid = txt[1].rstrip('\n').split(',')[-1]
        txt = [t.rstrip('\n').split(',') + [int(recordid)] for t in txt]
        txt_all.extend(txt[1:])
df_c = pd.DataFrame(txt_all, columns=['time', 'parameter', 'value', 'recordid'])
df_c = df_c[df_c["parameter"] != ""]

In [ ]:
df = pd.concat([df, df_c])
df.reset_index(inplace=True, drop=True) 

In [ ]:
len(df["recordid"].unique())

In [ ]:
def get_df_outcomes(dataset_outcomes):
    outcomes = pd.read_csv(dataset_outcomes)
    outcomes.set_index('RecordID', inplace=True)
    outcomes.index.name = 'recordid'
    outcomes = outcomes.reset_index()
    return outcomes

In [ ]:
df_outcomes_a = get_df_outcomes('Outcomes-a.txt')
df_outcomes_b = get_df_outcomes('Outcomes-b.txt')
df_outcomes_c = get_df_outcomes('Outcomes-c.txt')
df_outcomes = pd.concat([df_outcomes_a, df_outcomes_b, df_outcomes_c])
df_outcomes.reset_index(inplace=True, drop=True) 

### Filtering

In [ ]:
static_params = ["RecordID", "Age", "Gender", "Height", "ICUType", "Weight"]

In [ ]:
df["value"] = df["value"].apply(lambda x: float(x))
df["hour"] = df["time"].apply(lambda x: int(x.split(':')[0]))
df["minutes"] = df["time"].apply(lambda x: int(x.split(':')[1]))

In [ ]:
static_df = df.loc[df['time'] == '00:00', :].copy()

# retain only one of the 6 static vars:
static_params = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']
static_df = static_df.loc[df['parameter'].isin(static_params)]

In [ ]:
time_series_df = df.loc[~df.index.isin(static_df.index), :]

static_df = static_df.groupby(['recordid', 'parameter'])[['value']].last()
static_df.reset_index(inplace=True)

In [ ]:
static_df["recordid"] = static_df["recordid"].apply(lambda x: str(x))
static_df = static_df.pivot(index='recordid', columns='parameter', values='value')
static_df = static_df.reset_index()

#### encode categorical variables: ICUType and Gender to one-hot encoding

In [ ]:
# ICUType one-hot encoding
ICU_enc = OneHotEncoder()
ICU_enc.fit(static_df[["ICUType"]])
ICU_type_one_hot = ICU_enc.transform(static_df[["ICUType"]]).toarray()

In [ ]:
static_df["ICUType=1.0"]=ICU_type_one_hot[:,0]
static_df["ICUType=2.0"]=ICU_type_one_hot[:,1]
static_df["ICUType=3.0"]=ICU_type_one_hot[:,2]
static_df["ICUType=4.0"]=ICU_type_one_hot[:,3]

In [ ]:
# Gender one-hot encoding
# if gender value is -1.0, it is missing and we assign zero for all categories (female and male)
gender_enc = {0.0: np.eye(2)[0], 1.0: np.eye(2)[1], -1.0: np.array([0., 0.])}

In [ ]:
gender_enc_array = static_df["Gender"].apply(lambda x: gender_enc[x])

In [ ]:
gender_enc_array = np.stack(gender_enc_array.values)

In [ ]:
static_df["Gender=0.0"]=gender_enc_array[:,0]
static_df["Gender=1.0"]=gender_enc_array[:,1]

In [ ]:
time_series_df["recordid"] = time_series_df["recordid"].apply(lambda x: str(x))

### remove record ids that do not contain time series information

In [ ]:
recordids_to_remove = static_df[~static_df["recordid"].isin(time_series_df["recordid"].tolist())]["recordid"].unique().tolist()

In [ ]:
static_df = static_df[~static_df["recordid"].isin(recordids_to_remove)]
time_series_df = time_series_df[~time_series_df["recordid"].isin(recordids_to_remove)]

In [ ]:
static_df.shape

In [ ]:
def convert_time_to_decimal_time(t):
    t_strings = t.split(':')
    decimal_time = float(t_strings[0]) + float(t_strings[1])/60
    return round(decimal_time, 1)

In [ ]:
time_series_df["decimal_times"] = time_series_df["time"].apply(lambda x: convert_time_to_decimal_time(x))

In [ ]:
def get_hourly_time_groups(hour, mins):
    if mins in range(1, 60):
        new_time = str(hour + 1) + ":00"
    elif (hour == 0) and (mins == 0):
        new_time = str(hour + 1) + ":00"
    elif (hour != 0) and (mins == 0):
        new_time = str(hour) + ":00"
    return new_time

In [ ]:
time_series_df["time_group"] = time_series_df.apply(lambda x: get_hourly_time_groups(x["hour"], x["minutes"]), axis=1)

In [ ]:
time_series_params =  ["Albumin", "ALP", "ALT", "AST", "Bilirubin", "BUN", "Cholesterol", "Creatinine", \
                      "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "Mg", "MAP", \
                      "MechVent", "Na", "NIDiasABP", "NIMAP", "NISysABP", "PaCO2", "PaO2", "pH", "Platelets", \
                      "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight"]

In [ ]:
## aggregating the time steps to one hour bins
## if two or more measurements were taken within the same hour bin, we take the median

In [ ]:
timeseries_dict = defaultdict()
median_dict = time_series_df.groupby(["recordid", "time_group", "parameter"])["value"].median().to_dict()
for k, v in median_dict.items():
    timeseries_dict[k[0]] = defaultdict()

for k, v in median_dict.items():
    timeseries_dict[k[0]][k[1]] = defaultdict()

for k, v in median_dict.items():
    timeseries_dict[k[0]][k[1]][k[2]] = v

In [ ]:
final_df = static_df.copy()
final_df.pop("RecordID")

In [ ]:
timeseries_dict_GRU = defaultdict()
median_dict_GRU = time_series_df.groupby(["recordid", "decimal_times", "parameter"])["value"].median().to_dict()

for k, v in median_dict_GRU.items():
    timeseries_dict_GRU[k[0]] = defaultdict()

for k, v in median_dict_GRU.items():
    timeseries_dict_GRU[k[0]][k[1]] = defaultdict()

for k, v in median_dict_GRU.items():
    timeseries_dict_GRU[k[0]][k[1]][k[2]] = v

In [ ]:
def transform_input_per_time_group_GRU(record_dict, time):
    time_tracked = record_dict[time]
    param_list = list(time_tracked.keys())
    day_one_hot = [[time_tracked[t], 0.0] if t in param_list else [0.0, 1.0] for t in time_series_params]
    final = list(chain.from_iterable(day_one_hot))
    return final

In [ ]:
def transform_tracking_input_GRU(record_id):
    record_dict = timeseries_dict_GRU[record_id]
    tracking_times = list(record_dict.keys())
    
    final_list = [transform_input_per_time_group_GRU(record_dict, d) for d in tracking_times]
    return final_list

In [ ]:
GRU_input_tracking_one_hot = map(transform_tracking_input_GRU, static_df["recordid"].unique())
final_df["GRU_input_one_hot"] = list(GRU_input_tracking_one_hot)

In [ ]:
def get_GRU_times(record_id):
    record_dict = timeseries_dict_GRU[record_id]
    tracking_times = list(record_dict.keys())
    return tracking_times

In [ ]:
GRU_times = map(get_GRU_times, static_df["recordid"].unique())
final_df["GRU_times"] = list(GRU_times)

In [ ]:
time_intervals_to_keep = ['%s:00' % h for h in ([0] + list(range(1,49)))]
time_intervals_to_keep = time_intervals_to_keep[1:]

In [ ]:
def transform_input_per_time_group(record_dict, time_group):
    tracking_times = list(record_dict.keys())
    if time_group not in tracking_times:
        final = [0.0, 1.0] * 37
        return final
    elif time_group in tracking_times:
        time_tracked = record_dict[time_group]
        param_list = list(time_tracked.keys())
        day_one_hot = [[time_tracked[t], 0.0] if t in param_list else [0.0, 1.0] for t in time_series_params]
        final = list(chain.from_iterable(day_one_hot))
        return final

In [ ]:
def transform_tracking_input(record_id):
    if record_id in timeseries_dict.keys():
        record_dict = timeseries_dict[record_id]
        final_list = [transform_input_per_time_group(record_dict, d) for d in time_intervals_to_keep]
    else:
        final = [0.0, 1.0] * 37
        final_list = np.array(final * 48).reshape(48, 74)
        final_list = [list(x) for x in final_list]
    return final_list

In [ ]:
final_df["input_one_hot"] = static_df["recordid"].apply(lambda x: transform_tracking_input(x))

In [ ]:
df_outcomes["recordid"] = df_outcomes["recordid"].apply(lambda x: str(x))

In [ ]:
final_df = pd.merge(final_df, df_outcomes, how="left", left_on="recordid", right_on="recordid")

In [ ]:
final_df["In-hospital_death"].value_counts(normalize=True) * 100

In [ ]:
final_df["In-hospital_death"] = final_df["In-hospital_death"].apply(lambda x: int(x))

In [ ]:
class_balance_dict

### get missingness fractions

In [ ]:
def count_missing_features_per_timestep(input_one_hot):
    n_missing_features = []
    input_one_hot = np.array(input_one_hot)
    missing_matrix = input_one_hot[:,1::2]
    count_list = np.sum(missing_matrix, axis=1).tolist()
    return count_list

In [ ]:
final_df["n_missing_features"] = final_df["input_one_hot"].apply(lambda x: count_missing_features_per_timestep(x))

In [ ]:
# get percentage of missing values per input - output pair
final_df["total_missingness"] = final_df["n_missing_features"].apply(lambda x: sum(x))
n_features = 37
n_time_steps = 48
n_total = n_features * n_time_steps
final_df["total_frac_missingness"] = final_df["total_missingness"] / n_total

In [ ]:
final_df["total_frac_missingness"].mean() * 100

In [ ]:
final_df["total_frac_missingness"].min() * 100

In [ ]:
final_df["total_frac_missingness"].max() * 100

### prepare dataset for GRU-D

In [ ]:
def get_true_features(input_one_hot):
    input_one_hot = np.array(input_one_hot)
    true_features = input_one_hot[:,::2]
    return true_features

In [ ]:
def get_mask(input_one_hot):
    input_one_hot = np.array(input_one_hot)
    mask = 1 - input_one_hot[:,1::2]
    return mask

In [ ]:
final_df["X"] = final_df["GRU_input_one_hot"].apply(get_true_features)

In [ ]:
final_df["M"] = final_df["GRU_input_one_hot"].apply(get_mask)

In [ ]:
final_df["GRU_lengths"] = final_df["X"].apply(lambda x: len(x))

In [ ]:
final_df["tracked_sum_per_feature"] = final_df["M"].apply(lambda x: x.sum(axis=0))
final_df["feature_sum_per_patient"] =  final_df["X"].apply(lambda x: x.sum(axis=0)) 

In [ ]:
def replace_first_missing_with_mean(masks, values, mean_features_dict):
    # get indexes of the features that were not tracked 
    # where any first value is missing
    zero_idx = np.where(masks[0] == 0)[0]
    for idx in zero_idx:
        # replace first missing time step value of missing feature to empirical mean of that feature
        values[0,:][idx] = mean_features_dict[idx]
    return values

In [ ]:
def get_emperical_mean_features_dict(df):
    # get the emperical mean of the nonzero values in the dataset
    feature_sum_all_patients = np.stack(df["feature_sum_per_patient"])
    features_sum = feature_sum_all_patients.sum(axis=0)
    empirical_mean_features_dict = defaultdict()
    for i in range(0, 37):
        emp_mean = features_sum[i] / np.count_nonzero(feature_sum_all_patients[:,i])
        if np.isnan(emp_mean):
            emp_mean = 0.0
        empirical_mean_features_dict[i] = emp_mean
    return empirical_mean_features_dict

### preprocessing final data 

In [ ]:
def rescale_data_(x, input_mask, mean, std, encoder):
    """
    Rescale the dataset based on the mean and std of the training set
    Args:
        x: A np.array of several np.array with shape (t_i, d).
        mean: A np.array of shape (d,).
        std: A np.array of shape (d,).
    Returns:
        Same shape as x with rescaled values.
    """
    if encoder == "GRU":
        input_values = x[:,::2]
        input_mask = 1 - x[:,1::2]
        # have np.nan instead of 0
        mdata = ma.masked_array(input_values, mask=~input_mask.astype(bool))
    elif encoder == "GRUD":
        mdata = ma.masked_array(x, mask=~input_mask.astype(bool))
    r = np.asarray([(xx - mean[np.newaxis, :]) / std[np.newaxis, :] for xx in mdata])
    r = r.reshape(mdata.shape)
    
    if encoder == "GRU":
        x[:,::2] = r
        return x
    elif encoder == "GRUD":
        return r

In [ ]:
def get_train_mean_std(x):
    input_values = x[:,:,::2]
    input_mask = 1 - x[:,:,1::2]
    mdata = ma.masked_array(input_values, mask=~input_mask.astype(bool))
    train_std = mdata.std(axis=(0, 1))
    mean_std = mdata.mean(axis=(0, 1))
    return mean_std, train_std

In [ ]:
new_static_params = ["RecordID", "Age", "Gender=0.0", "Gender=1.0", \
                     "Height", "ICUType=1.0", "ICUType=2.0", "ICUType=3.0", "ICUType=4.0", "Weight"]
static_input_cols = new_static_params[1:]

In [ ]:
# split dataset into train, validation, and test set
train, test = train_test_split(final_df, test_size=0.2, stratify=final_df["In-hospital_death"], random_state=21)
train, val = train_test_split(train, test_size=0.2, stratify=train["In-hospital_death"], random_state=21)

In [ ]:
# get the emperical mean from the training dataset
train_emperical_mean_dict = get_emperical_mean_features_dict(train)

In [ ]:
# for GRU-D: update first value with the empirical mean if the first value is missing
train_x_updated_first_values = train.apply(lambda x: replace_first_missing_with_mean(x["M"], x["X"], train_emperical_mean_dict), axis=1)
val_x_updated_first_values = val.apply(lambda x: replace_first_missing_with_mean(x["M"], x["X"], train_emperical_mean_dict), axis=1)
test_x_updated_first_values = test.apply(lambda x: replace_first_missing_with_mean(x["M"], x["X"], train_emperical_mean_dict), axis=1)

In [ ]:
# missing values of height and weight are replaced by the mean of the training dataset
train_aux_df = train[["Height", "Weight"]]
train_aux_df = train_aux_df.replace(-1, np.nan)
train_mean = train_aux_df.mean()
train[["Height", "Weight"]] = train_aux_df.fillna(train_mean)

val_aux_df = val[["Height", "Weight"]]
val_aux_df = val_aux_df.replace(-1, np.nan)
val[["Height", "Weight"]] = val_aux_df.fillna(train_mean)

test_aux_df = test[["Height", "Weight"]]
test_aux_df = test_aux_df.replace(-1, np.nan)
test[["Height", "Weight"]] = test_aux_df.fillna(train_mean)

### standardize final dataset

In [ ]:
data = defaultdict()
for split in ["train", "val", "test"]:
    data[split] = {}

### train

In [ ]:
#prepare training set
X_train = np.stack(train["input_one_hot"])
#mean: A np.array of shape (d,)
#std: A np.array of shape (d,)
mean_train, std_train = get_train_mean_std(X_train) 

# rescale non-missing values to non-zero mean
# prepare GRU input data
data["train"]["X_train"] = np.array([rescale_data_(x, 0, mean_train, std_train, "GRU") for x in X_train])

# prepare GRUD input data
X_train_GRUD = np.array(train["X"])
data["train"]["M"] = np.array(train["M"])
data["train"]["X"] = np.array([rescale_data_(x, data["train"]["M"][i], mean_train, std_train, "GRUD") for i, x in enumerate(X_train_GRUD)])

data["train"]["GRU_lengths"] = np.array(train["GRU_lengths"])
data["train"]["GRU_times"] = np.array(train["GRU_times"])

# standardize static input
X_train_aux = train[static_input_cols]
scaler = StandardScaler().fit(X_train_aux)
data["train"]["X_aux"] = scaler.transform(X_train_aux)

# turn output classes into categorical one-hot-encoding
data["train"]["y"] = np_utils.to_categorical(np.array(train["In-hospital_death"]), num_classes=2).astype(float)
data["train"]["y_classes"] = np.array(train["In-hospital_death"])

### validation

In [ ]:
#prepare validation set
X_val = np.stack(val["input_one_hot"])
data["val"]["X_val"] = np.array([rescale_data_(x, 0, mean_train, std_train, "GRU") for x in X_val])

# prepare GRUD input data
X_val_GRUD = np.array(val["X"])
data["val"]["M"] = np.array(val["M"])
data["val"]["X"] = np.array([rescale_data_(x, data["val"]["M"][i], mean_train, std_train, "GRUD") for i, x in enumerate(X_val_GRUD)])

data["val"]["GRU_lengths"] = np.array(val["GRU_lengths"])
data["val"]["GRU_times"] = np.array(val["GRU_times"])

# standardize static input
X_val_aux = val[static_input_cols]
data["val"]["X_aux"] = scaler.transform(X_val_aux)

# turn output classes into categorical one-hot-encoding
data["val"]["y"] = np_utils.to_categorical(np.array(val["In-hospital_death"]), num_classes=2).astype(float)
data["val"]["y_classes"] = np.array(val["In-hospital_death"])

### test

In [ ]:
#prepare test set
X_test = np.stack(test["input_one_hot"])
data["test"]["X_test"] = np.array([rescale_data_(x, 0, mean_train, std_train, "GRU") for x in X_test])

# prepare GRUD input data
X_test_GRUD = np.array(test["X"])
data["test"]["M"] = np.array(test["M"])
data["test"]["X"] = np.array([rescale_data_(x, data["test"]["M"][i], mean_train, std_train, "GRUD") for i, x in enumerate(X_test_GRUD)])

data["test"]["GRU_lengths"] = np.array(test["GRU_lengths"])
data["test"]["GRU_times"] = np.array(test["GRU_times"])

# standardize static input
X_test_aux = test[static_input_cols]
data["test"]["X_aux"] = scaler.transform(X_test_aux)

# turn output classes into categorical one-hot-encoding
data["test"]["y"] = np_utils.to_categorical(np.array(test["In-hospital_death"]), num_classes=2).astype(float)
data["test"]["y_classes"] = np.array(test["In-hospital_death"])

### save final dataset

In [ ]:
with open('physionet2012.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)